In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from pprint import pprint
import random
import json
from scraping_utils import scrape_resume, scrape_resume_links
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
categories = [
    "https://www.hireitpeople.com/resume-database/66-business-analyst-resumes",
    "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
    "https://www.hireitpeople.com/resume-database/71-sap-resumes",
    "https://www.hireitpeople.com/resume-database/73-datawarehousing-etl-informatica-resumes",
    "https://www.hireitpeople.com/resume-database/74-business-intelligence-business-object-resumes",
    "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
    "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
    "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes",
    "https://www.hireitpeople.com/resume-database/81-project-manager-resumes",
    "https://www.hireitpeople.com/resume-database/82-quality-assurance-resumes",
    "https://www.hireitpeople.com/resume-database/87-sql-developers-resumes"
]


# "https://www.hireitpeople.com/resume-database/63-net-developers-architects-resumes",
# "https://www.hireitpeople.com/resume-database/64-java-developers-architects-resumes",
# "https://www.hireitpeople.com/resume-database/65-informatica-developers-architects-resumes",
# "https://www.hireitpeople.com/resume-database/66-business-analyst-resumes",
# "https://www.hireitpeople.com/resume-database/67-quality-assurance-qa-resumes",
# "https://www.hireitpeople.com/resume-database/68-network-and-systems-administrators-resumes",
# "https://www.hireitpeople.com/resume-database/69-help-desk-support-resumes",
# "https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes",
# "https://www.hireitpeople.com/resume-database/71-sap-resumes",
# "https://www.hireitpeople.com/resume-database/72-web-developer-resumes",
# "https://www.hireitpeople.com/resume-database/73-datawarehousing-etl-informatica-resumes",
# "https://www.hireitpeople.com/resume-database/74-business-intelligence-business-object-resumes",
# "https://www.hireitpeople.com/resume-database/75-mainframe-resumes",
# "https://www.hireitpeople.com/resume-database/76-network-admin-resumes",
# "https://www.hireitpeople.com/resume-database/77-oracle-resumes",
# "https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes",
# "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes",
# "https://www.hireitpeople.com/resume-database/81-project-manager-resumes",
# "https://www.hireitpeople.com/resume-database/82-quality-assurance-resumes",
# "https://www.hireitpeople.com/resume-database/83-recruiter-resumes",
# "https://www.hireitpeople.com/resume-database/85-sas-resumes",
# "https://www.hireitpeople.com/resume-database/86-sharepoint-resumes",
# "https://www.hireitpeople.com/resume-database/87-sql-developers-resumes",
# "https://www.hireitpeople.com/resume-database/88-technical-writers-resumes",
# "https://www.hireitpeople.com/resume-database/89-websphere-resumes",

num_pages = 500    # each page contains 10 resumes

In [4]:
all_links = {}

with ThreadPoolExecutor(max_workers=len(categories)) as executor:
    futures = {executor.submit(scrape_resume_links, cat_url, max_pages=500): cat_url for cat_url in categories}
    for future in as_completed(futures):
        cat_url = futures[future]
        try:
            links = future.result()
            all_links[cat_url] = links
            print(f"[✓] Collected {len(links)} resumes from {cat_url}")
        except Exception as e:
            print(f"[!] Failed to scrape {cat_url}: {e}")

# flatten list
all_resume_links = [link for links in all_links.values() for link in links]

[+] Scraping https://www.hireitpeople.com/resume-database/66-business-analyst-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/70-oracle-developers-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/71-sap-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/73-datawarehousing-etl-informatica-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/74-business-intelligence-business-object-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/77-oracle-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/78-oracle-dba-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/81-project-manager-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/82-quality-assurance-resumes/page/1
[+] Scraping https://www.hireitpeople.com/resume-database/87-sql-dev

In [5]:
session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0 ..."})

org_resume_dict = {}

print(f"[+] Scraping {len(all_resume_links)} resumes in parallel...")

with ThreadPoolExecutor(max_workers=12) as executor:  # adjust workers
    futures = {executor.submit(scrape_resume, url, session): url for url in all_resume_links}
    
    # tqdm progress bar with total count
    for future in tqdm(as_completed(futures), total=len(futures), desc="Scraping resumes", unit="resume"):
        url = futures[future]
        try:
            data = future.result()
            org_resume_dict[url] = data
        except Exception as e:
            print(f"[!] Failed {url}: {e}")

print(f"[✔] Completed scraping {len(org_resume_dict)} resumes.")

[+] Scraping 49730 resumes in parallel...


Scraping resumes:   0%|                             | 8/49730 [00:06<8:29:42,  1.63resume/s]

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/627034-lead-oracle-database-administrator-resume-highlands-ranch-co-1: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/626958-account-tech-subject-matter-expert-tech-lead-resume-5: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Scraping resumes:   0%|                            | 40/49730 [00:18<5:59:19,  2.30resume/s]

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/624509-oracle-financial-functional-consultant-resume-chicago-il-1: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


...

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/604485-oracle-developer-resume-albany-ny-4: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Scraping resumes:   0%|                           | 156/49730 [01:03<4:41:35,  2.93resume/s]

[!] Failed https://www.hireitpeople.com/resume-database/77-oracle-resumes/588260-oracle-fusion-middleware-administrator-resume-sacramento-california-3: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Scraping resumes: 100%|███████████████████████████| 49730/49730 [37:46<00:00, 21.94resume/s]

[✔] Completed scraping 49719 resumes.


In [6]:
with open("resumes.json", "w", encoding="utf-8") as f:
    json.dump(org_resume_dict, f, ensure_ascii=False, indent=4)

In [7]:
df = pd.DataFrame.from_dict(org_resume_dict, orient="index").reset_index()
df.rename(columns={"index": "url"}, inplace=True)

# Convert any dict/list column into a JSON string
for col in df.columns:
    df[col] = df[col].apply(
        lambda x: json.dumps(x, ensure_ascii=False) if isinstance(x, (dict, list)) else x
    )

# Save to Parquet
df.to_parquet("resumes.parquet", engine="pyarrow", index=False)
print("✅ Saved to resumes.parquet")


✅ Saved to resumes.parquet


In [23]:
# all_links = {}
# for cat_url in categories:
#     links = scrape_resume_links(cat_url, max_pages=1)
#     all_links[cat_url] = links
#     print(f"[✓] Collected {len(links)} resumes from {cat_url}")


# with open("resume_links.txt", "w") as f:
#     for cat, links in all_links.items():
#         f.write(f"# {cat}\n")
#         for link in links:
#             f.write(link + "\n")



# org_resume_dict = {}

# for domain_link in all_links.keys():
#     domain_resume_list = []
#     for resume_link in all_links[domain_link]: 
#         resume_dict = scrape_resume(resume_link)
#         domain_resume_list.append(resume_dict)
#         print(f"[✓] {resume_link}")
#     org_resume_dict[domain_link] = domain_resume_list
#     print(f"DOMAIN COMPLETED: {domain_link}")
    